### **Lab Exercise Overview**

In this lab exercise, you will create a Django application using the django framework. In the following tasks, you will add a user registration workflow to enable users to register and activate new accounts.

**Prerequisites**

* Linux environment (If you are on Windows, follow the instructions here https://docs.microsoft.com/en-us/windows/wsl/install to install the Windows Subsystem for Linux)
* python3, pip installed
* latest version of Django
* an existing Django project

#### **Task 1: Add built-in authentication views to your project**

* Modify the URL path entries in **settings.py** to include the following entry:

    > ``urlpatterns = [...path('accounts/',include('django.contrib.auth.urls'))]``<br/>
<br/> The following mapping is applied in your Django project:<br/> 
        * accounts/login/ LoginView <br/>
        * accounts/logout/ LogoutView<br/>
        * accounts/password_change/ PasswordChangeView<br/>
        * accounts/password_change/done/ PasswordChangeDoneView<br/>
        * accounts/password_reset/ PasswordResetView<br/>
        * accounts/password_reset/done/ PasswordResetDoneView<br/>
        * accounts/reset/<uidb64>/<token>/ PasswordResetConfirmView<br/>
        * accounts/reset/done/ PasswordResetCompleteView

<br/>

#### **Task 2: Create a Django application for user authentication**

* Open your Django project directory
* Run the following command to create a new Django application
    > ``python manage.py startapp profile_info``
* Open **views.py** in the profile_info root directory
* Add the following code to access the built-in user object in the ProfileView:
    > ```python
        from django.shortcuts import render
        from django.http import HttpResponse
        from django.views.generic import View

        class ProfileView(View):
            def get(self, request):
                user = request.user
                if not user.is_authenticated:
                    return HttpResponse(status=401)
                return render(request,'profile.html')
    ```
* Add a new file called **urls.py** in the profile_info root directory and add the following code:
    > ```python
        import views
        from django.urls import path

        urlpatterns = [path('profile/', views.ProfileView.as_view(),name='profile'),]
    ```
* In the project root directory reopen urls.py and add the folling URL path entry to the **urlpatterns** list:
    > ```python
        path('accounts/', include('django.contrib.auth.urls')), 
        path('accounts/', include('profile_info.urls'))
    ```
* Add a new subdirectory name **registration** in the templates directory
* In the **registration** directory, create a new file named **login.html** and add the following code:
    > ```html
        {% load bootstrap5 %}
        {% bootstrap_css %}
        {% bootstrap_javascript %}
        {% bootstrap_messages %}
        <div class="container">
            <h1 class="display-4">Login</h1>
            <hr/>
            <form  method="post" class="form">
            {% csrf_token %}
            {% bootstrap_form form %}
            {% buttons %}
                <button type="submit" class="btn btn-primary">
                Login
                </button>
            {% endbuttons %}
            </form>
        </div>
    ```
* In the **templates** directory, create a new file named **profile.html** and add the following code:
    >```html
        {% load bootstrap5 %}
        {% bootstrap_css %}
        {% bootstrap_javascript %}
        {% bootstrap_messages %}
        <div class="container">
            <p>
            Hello {{ user.username }}, 
            your email is {{ user.email }}. 
            </p>
            <p>
            <a href="{% url 'logout' %}">Logout</a> 
            </p>
        </div>
    ```
* Save your changes and restart the server:
    > ``gunicorn simpleapp.wsgi --keyfile private_key.pem --certfile certificate.pem``

* Open https://localhost:8000/accounts/login/
* Open the Developer Tools in your browser and navigate to the Network tab
* Enter the username and password you created for the user and press Login
* In the Network tab, you will see that the response headers includes Set-Cookie which has the sessionId
* In the request headers of accounts/profile you will also find the sessionId in the cookie header


#### Prohibit anonymous access

You can use the **LoginRequiredMixin** class to prohibit anonymous access to protected application resources. You will modify the **ProfileView** class to make use of the **LoginRequiredMixin**.

>```python
        from django.shortcuts import render
        from django.contrib.auth.mixins import LoginRequiredMixin
        from django.views.generic import View

        class ProfileView(LoginRequiredMixin,View):
            def get(self, request):
                return render(request,'profile.html')```
<br/>

#### Test Login

* Create and open a file name **tests.py** under the profile_info directory
* Create a new class named **TestAuthentication** that inherits from django.test.TestCase
* Add the following code to the class:
    > ```python
        from django.contrib.auth import get_user_model
        from django.test import TestCase

        class TestAuthentication(TestCase):
            def test_authenticated_workflow(self):
                passphrase = 'wool reselect resurface annuity' 
                get_user_model().objects.create_user('bob', password=passphrase) 
                self.client.login(username='bob', password=passphrase) 
                self.assertIn('sessionid', self.client.cookies) 
                response = self.client.get( 
                '/accounts/profile/', 
                secure=True) 
                self.assertEqual(200, response.status_code) 
                self.assertContains(response, 'bob') 
                self.client.logout() 
                self.assertNotIn('sessionid', self.client.cookies) 
        
            def test_prohibit_anonymous_access(self):
                response = self.client.get('/accounts/profile/', secure=True) 
                self.assertEqual(302, response.status_code) 
                self.assertIn('/accounts/login/', response['Location'])```

* From the project root directory, run ``python manage.py test`` and note down the result

